In [3]:
# You must restart the runtime in order to use newly installed versions.
!pip install -U pip setuptools wheel
!pip install -U spacy

  Using cached spacy-3.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
  Using cached blis-1.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.6 kB)
Using cached spacy-3.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.1 MB)
Using cached thinc-8.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.7 MB)
Using cached blis-1.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.2 MB)
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Succes

In [2]:
# Restart to reload dependencies.
!python -m spacy download ru_core_news_lg
!python -m spacy download en_core_web_trf
!python -m spacy download zh_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 22.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  

In [1]:
import re
import pandas as pd
import string
import spacy
model_ru = spacy.load('ru_core_news_lg')
model_en = spacy.load('en_core_web_trf')
model_zh = spacy.load('zh_core_web_trf')

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

In [2]:
source_lang = 'en'
valid_source_lang_codes = ['ru', 'en', 'zh']

In [3]:
allowed_abbr_patterns = r"\([а-яa-z]*[А-ЯA-Z]{2,}[а-яa-z]*[-а-яa-zА-ЯA-Z]*\)|[а-яa-z]*[А-ЯA-Z]{2,}[а-яa-z]*[-а-яa-zА-ЯA-Z]* \(.*\)"
allowed_characters_enru = r"^[а-яa-zА-ЯA-Z \(\)—–-]+$"

In [11]:
#ru
#extracted_terms = [{'id': 0, 'text': 'Название Учреждения'}, {'id': 1, 'text': 'Версия 1.0 от 20.10.2015 г.'}, {'id': 2, 'text': 'management system'}, {'id': 3, 'text': 'система менеджмента качества (принятая на производстве)'}, {'id': 4, 'text': 'система (принятая на производстве) менеджмента качества'}, {'id': 5, 'text': 'название модели: Turbo'}, {'id': 6, 'text': 'название отдела, департамента или подразделения'}, {'id': 7, 'text': 'контр-адмирал'}, {'id': 8, 'text': 'REV-STEMI-III'}, {'id': 9, 'text': 'тромб'}, {'id': 10, 'text': '«Берингер Ингельхайм Фарма» ГмбХ и КоКГ'}, {'id': 11, 'text': 'история болезни'}, {'id': 12, 'text': 'артериальное давление (АД)'}, {'id': 13, 'text': 'The source segments do not contain any terms to extract. They are all URLs or an email address, which do not represent specific terms or concepts.'}, {'id': 14, 'text': 'лиофилизат'}, {'id': 15, 'text': 'ЗАО «ГЕНЕРИУМ»'}, {'id': 16, 'text': 'на момент выписки'}, {'id': 17, 'text': 'антиагреганты'}, {'id': 18, 'text': 'антикоагулянты'}, {'id': 19, 'text': 'Пневмония'}, {'id': 20, 'text': 'неврологический дефицит'}, {'id': 21, 'text': 'III Фаза'}, {'id': 22, 'text': 'эндоваскулярные устройства'}, {'id': 23, 'text': 'Шкала NIHSS'}, {'id': 24, 'text': 'внутривенная сТЛТ (системная тромболитическая терапия)'}, {'id': 25, 'text': 'Симптомная геморрагическая трансформация'}, {'id': 26, 'text': 'систолическое АД (артериальное давление)'}, {'id': 27, 'text': 'clinicaltrials.gov'}, {'id': 28, 'text': 'механическая тромбэкстракция'}, {'id': 29, 'text': 'специализированные стационары'}, {'id': 30, 'text': 'Коронавирусная инфекция'}, {'id': 31, 'text': 'функциональные классы (ФК)'}, {'id': 32, 'text': '3—6 баллов MUP'}, {'id': 33, 'text': 'функциональное восстановление'}, {'id': 34, 'text': 'геморрагическая трансформация очага поражения головного мозга'}, {'id': 35, 'text': '№ КИ-20/13, версия № 4 от 15.12.2014 г'}, {'id': 36, 'text': 'тромбоз церебральных артерий'}, {'id': 37, 'text': 'терапия пациентов с ИИ'}, {'id': 38, 'text': 'последствия перенесенного ИИ'}, {'id': 39, 'text': 'инвалидизация'}, {'id': 40, 'text': 'сТЛТ (системная тромболитическая терапия)'}, {'id': 41, 'text': '0—2 балла MUIP'}, {'id': 42, 'text': 'Функциональное восстановление'}, {'id': 43, 'text': 'неврология'}, {'id': 44, 'text': 'закупорка церебральной артерии'}, {'id': 45, 'text': 'гипертоническая болезнь'}, {'id': 46, 'text': 'системная тромболитическая терапия (сТЛТ)'}, {'id': 47, 'text': 'МШР 0—2'}, {'id': 48, 'text': 'безопасность'}, {'id': 49, 'text': 'Коморбидность'}, {'id': 50, 'text': 'функциональная независимость'}, {'id': 51, 'text': 'тромб'}, {'id': 52, 'text': 'МШР (модифицированная шкала Рэнкина)'}, {'id': 53, 'text': 'телефонные контакты.'}, {'id': 54, 'text': 'Генеральная Ассамблея Всемирной медицинской ассоциации'}, {'id': 55, 'text': 'цереброваскулярные болезни'}, {'id': 56, 'text': 'первичные сосудистые отделения (ПСО)'}, {'id': 57, 'text': 'COVID-19'}, {'id': 58, 'text': 'Федеральный закон РФ от 12.04.2010 №61-ФЗ'}, {'id': 59, 'text': 'Через 1 ч после проведения сТЛТ'}, {'id': 60, 'text': 'глобальное бремя болезней'}, {'id': 61, 'text': 'параллельные группы'}, {'id': 62, 'text': 'искусственная вентиляция легких (ИВЛ)'}, {'id': 63, 'text': 'внутривенная сТЛТ'}, {'id': 64, 'text': 'Пандемия'}, {'id': 65, 'text': 'антропометрические измерения'}, {'id': 66, 'text': 'больничная летальность'}, {'id': 67, 'text': 'Время обследования'}, {'id': 68, 'text': 'Антикоагулянты'}, {'id': 69, 'text': 'Лабораторное тестирование'}, {'id': 70, 'text': 'Летальность'}, {'id': 71, 'text': 'Антиагреганты'}, {'id': 72, 'text': '90±7 дней после проведения сТЛТ'}, {'id': 73, 'text': 'На 7-е сутки'}, {'id': 74, 'text': '30±7 дней после проведения сТЛТ'}, {'id': 75, 'text': 'диастолическое АД (артериальное давление)'}, {'id': 76, 'text': 'глюкоза крови'}, {'id': 77, 'text': 'Правила клинической практики'}, {'id': 78, 'text': 'приказ Минздрава России от 01.04.2016 №200н'}, {'id': 79, 'text': 'пандемия COVID-19'}, {'id': 80, 'text': 'отдаленном периоде'}, {'id': 81, 'text': 'МШР'}, {'id': 82, 'text': 'Доля пациентов с неврологическим улучшением'}, {'id': 83, 'text': 'регресс неврологических симптомов'}, {'id': 84, 'text': 'Через 1 сут после проведения сТЛТ'}, {'id': 85, 'text': 'Шкала МШР'}, {'id': 86, 'text': 'АО «ГЕНЕРИУМ»'}, {'id': 87, 'text': 'модифицированная шкала Рэнкина (МШР)'}, {'id': 88, 'text': 'ст. 21 Конституции РФ'}, {'id': 89, 'text': 'Федеральный закон РФ от 21.11.2011 №323-ФЗ'}, {'id': 90, 'text': 'Вирус SARS-CoV-2'}, {'id': 91, 'text': 'эмбол'}, {'id': 92, 'text': 'симптомная геморрагическая трансформация'}, {'id': 93, 'text': 'Национальный стандарт РФ ГОСТ Р 52379-2005'}, {'id': 94, 'text': 'Постгоспитальный период'}, {'id': 95, 'text': 'Хельсинкская декларация'}, {'id': 96, 'text': 'При поступлении в стационар'}, {'id': 97, 'text': 'Ученый совет ФГБУ ФЦМН ФМБА России'}, {'id': 98, 'text': 'медицинская эвакуация'}, {'id': 99, 'text': 'Факторы риска развития ИИ'}, {'id': 100, 'text': 'Медицинская онлайн-платформа ROSMED.INFO'}, {'id': 101, 'text': 'реперфузионная терапия'}, {'id': 102, 'text': 'шкала NIHSS'}]

In [12]:
#zh
#extracted_terms = [{'id': 0, 'text': '贵,宾犬'}, {'id': 1, 'text': '松狮犬'}, {'id': 2, 'text': '牧羊犬'}, {'id': 3, 'text': '斗牛狗 (斗牛狗)'}, {'id': 4, 'text': '拳师狗'}, {'id': 5, 'text': '杜宾犬: Turbo'}, {'id': 6, 'text': '英 国 古 代 牧 羊 犬'}, {'id': 7, 'text': '圣伯纳狗'}, {'id': 8, 'text': '动物'}, {'id': 9, 'text': '鱼'}, {'id': 10, 'text': '猫'}, {'id': 11, 'text': '大象'}]

In [13]:
#en
extracted_terms = [
    {'id': 1, 'text': 'loading/unloading station'},
    {'id': 2, 'text': 'planned dates of delivery for loading'},
    {'id': 3, 'text': 'installation of locking and sealing devices on cars'},
    {'id': 4, 'text': 'financial lease (leasing)'},
    {'id': 5, 'text': "payer's bank account"},
    {'id': 6, 'text': 'certificate of an independent surveyor company'},
    {'id': 7, 'text': 'planned dates of delivery for loading'},
    {'id': 8, 'text': 'pre-trial claim procedure for dispute settlement'},
    {'id': 9, 'text': 'Phase III'},
    {'id': 10, 'text': 'antiplatelets'},
    {'id': 11, 'text': 'blood pressure (BP)'},
    {'id': 12, 'text': 'proportion for patients with neurological improvement'},
    {'id': 13, 'text': 'cerebral artery obstruction'},
    {'id': 14, 'text': 'mechanical ventilation'},
    {'id': 15, 'text': 'mRS (modified Rankin Scale)'},
    {'id': 16, 'text': 'departments of vascular surgery (DVSs)'},
    {'id': 17, 'text': 'treatment of patients with IS'},
    {'id': 18, 'text': 'functional classes (FC)'},
    {'id': 19, 'text': 'the functional class'},
    {'id': 20, 'text': 'official'},
    ]

In [14]:
def dealing_with_brackets(extracted_terms):
# Process terms with parentheses
# - Add terms matching the patterns "abbreviation (expansion)" or "expansion (abbreviation)" to the glossary; abbreviations can be in forms such as "AD," "cTLT," or "cTLT-Pr."
# - In all other cases, do not add terms containing parentheses to the glossary.
# - Add terms without parentheses unchanged.
    extracted_terms_brackets = []
    for term in extracted_terms:
        if re.search(r"\(.*\)", term['text']):
            if re.search(allowed_abbr_patterns, term['text']):
                extracted_terms_brackets.append(term)
        else:
            extracted_terms_brackets.append(term)
    return(extracted_terms_brackets)

In [15]:
def filter_extracted_terms_by_ner(extracted_terms, source_lang):
# Remove terms with named entities
    if source_lang == 'ru':
        model = model_ru
    elif source_lang == 'en':
        model = model_en
    elif source_lang == 'zh':
        model = model_zh
    filtered_extracted_terms = []
    for i in extracted_terms:
        if len(model(i['text']).ents) == 0:
            filtered_extracted_terms.append(i)
    return filtered_extracted_terms

In [16]:
def cleaning_extracted_terms(extracted_terms, source_lang):
    if source_lang in valid_source_lang_codes:
        extracted_terms_wo_ents = filter_extracted_terms_by_ner(extracted_terms, source_lang)
        print(extracted_terms_wo_ents)
        extracted_terms_brackets = dealing_with_brackets(extracted_terms_wo_ents)
        print(extracted_terms_brackets)
        # To do: add additional cleaning for Chinese in the future
        if source_lang == 'zh':
            return(extracted_terms_brackets)
        else:
            # Clean glossary inclusion criteria
            # Add only terms containing Cyrillic or Latin characters, spaces, and the symbols '—', '–', and '-' to the clean glossary. Additionally, remove the following:
            # - Terms that do not contain any letters from the source language
            # - Terms with more than 5 words in Russian or more than 4 words in English
            # - Terms in English that contain the preposition 'of'
            # - Terms where all letters are uppercase
            # - Duplicates
            clean_extracted_terms = []
            for term in extracted_terms_brackets:
                if (term['text'].isupper() == False and re.search(allowed_characters_enru, term['text']) and
                    term['text'].strip() not in [d['text'] for d in clean_extracted_terms]):
                    if (source_lang == 'ru' and re.search(r"^[^а-яА-Я]+$", term['text']) == None and
                        term['text'].strip().count(' ') < 5):
                        clean_extracted_terms.append(term)
                    if (source_lang == 'en' and re.search(r"^[^a-zA-Z]+$|\bof\b", term['text']) == None and
                        term['text'].strip().count(' ') < 4):
                        clean_extracted_terms.append(term)
            return(clean_extracted_terms)
    else:
      empty_gloss = []
      return(empty_gloss)

In [17]:
clean_extracted_terms = cleaning_extracted_terms(extracted_terms, source_lang)
clean_extracted_terms

[{'id': 1, 'text': 'loading/unloading station'}, {'id': 2, 'text': 'planned dates of delivery for loading'}, {'id': 3, 'text': 'installation of locking and sealing devices on cars'}, {'id': 4, 'text': 'financial lease (leasing)'}, {'id': 5, 'text': "payer's bank account"}, {'id': 6, 'text': 'certificate of an independent surveyor company'}, {'id': 7, 'text': 'planned dates of delivery for loading'}, {'id': 8, 'text': 'pre-trial claim procedure for dispute settlement'}, {'id': 10, 'text': 'antiplatelets'}, {'id': 11, 'text': 'blood pressure (BP)'}, {'id': 12, 'text': 'proportion for patients with neurological improvement'}, {'id': 13, 'text': 'cerebral artery obstruction'}, {'id': 14, 'text': 'mechanical ventilation'}, {'id': 15, 'text': 'mRS (modified Rankin Scale)'}, {'id': 16, 'text': 'departments of vascular surgery (DVSs)'}, {'id': 17, 'text': 'treatment of patients with IS'}, {'id': 18, 'text': 'functional classes (FC)'}, {'id': 19, 'text': 'the functional class'}, {'id': 20, 'tex

[{'id': 10, 'text': 'antiplatelets'},
 {'id': 11, 'text': 'blood pressure (BP)'},
 {'id': 13, 'text': 'cerebral artery obstruction'},
 {'id': 14, 'text': 'mechanical ventilation'},
 {'id': 15, 'text': 'mRS (modified Rankin Scale)'},
 {'id': 18, 'text': 'functional classes (FC)'},
 {'id': 19, 'text': 'the functional class'},
 {'id': 20, 'text': 'official'}]

In [18]:
# Check which terms were removed and which ones remain.
list_of_deleted_terms = []

for term in extracted_terms:
  if term['text'] not in [d['text'] for d in clean_extracted_terms]:
    list_of_deleted_terms.append([term, 'deleted'])
  else:
    list_of_deleted_terms.append([term, term])

list_of_deleted_terms

[[{'id': 1, 'text': 'loading/unloading station'}, 'deleted'],
 [{'id': 2, 'text': 'planned dates of delivery for loading'}, 'deleted'],
 [{'id': 3, 'text': 'installation of locking and sealing devices on cars'},
  'deleted'],
 [{'id': 4, 'text': 'financial lease (leasing)'}, 'deleted'],
 [{'id': 5, 'text': "payer's bank account"}, 'deleted'],
 [{'id': 6, 'text': 'certificate of an independent surveyor company'},
  'deleted'],
 [{'id': 7, 'text': 'planned dates of delivery for loading'}, 'deleted'],
 [{'id': 8, 'text': 'pre-trial claim procedure for dispute settlement'},
  'deleted'],
 [{'id': 9, 'text': 'Phase III'}, 'deleted'],
 [{'id': 10, 'text': 'antiplatelets'}, {'id': 10, 'text': 'antiplatelets'}],
 [{'id': 11, 'text': 'blood pressure (BP)'},
  {'id': 11, 'text': 'blood pressure (BP)'}],
 [{'id': 12, 'text': 'proportion for patients with neurological improvement'},
  'deleted'],
 [{'id': 13, 'text': 'cerebral artery obstruction'},
  {'id': 13, 'text': 'cerebral artery obstruction